In [1]:
!pip install nnfs

In [2]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights ) + self.biases
        return self.output

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis = 0, keepdims = True)
        self.dinputs = np.dot(dvalues, self.weights.T)

In [3]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)
        return self.output

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self, inputs):
        exponantial = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        normalize = exponantial / np.sum(exponantial, axis=1, keepdims=True)
        self.output = normalize
        return self.output

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)

        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1,1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix,single_dvalues)


In [4]:
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss


class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]

        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped*y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
    
        if len(y_true.shape) == 1:
            y_true = np.eye(dvalues.shape[1])[y_true]
    
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


In [5]:
class Activation_Softmax_Loss_CategoricalCrossentropy(): 
 
    def __init__(self): 
        self.activation = Activation_Softmax() 
        self.loss = Loss_CategoricalCrossentropy() 
  
    def forward(self, inputs, y_true): 
        self.activation.forward(inputs) 
        self.output = self.activation.output 
        return self.loss.calculate(self.output, y_true) 
 
    def backward(self, dvalues, y_true): 
        samples = len(dvalues) 
 
        if len(y_true.shape) == 2: 
            y_true = np.argmax(y_true, axis=1) 
 
        self.dinputs = dvalues.copy() 
        self.dinputs[range(samples), y_true] -= 1 
        self.dinputs = self.dinputs / samples

In [6]:
import numpy as np 
import nnfs 
from nnfs.datasets import spiral_data 
 
nnfs.init() 

X, y = spiral_data(samples=100, classes=3) 

dense1 = Layer_Dense(2, 3) 
activation1 = Activation_ReLU() 
dense2 = Layer_Dense(3, 3) 
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy() 
 
dense1.forward(X)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
loss = loss_activation.forward(dense2.output, y)



print(loss_activation.output[:5]) 

print('loss:', loss) 

predictions = np.argmax(loss_activation.output, axis=1) 
if  len(y.shape) ==  2: 
   y =  np.argmax(y, axis=1) 
accuracy = np.mean(predictions == y) 

print('acc:', accuracy) 

loss_activation.backward(loss_activation.output, y) 
dense2.backward(loss_activation.dinputs) 
activation1.backward(dense2.dinputs) 
dense1.backward(activation1.dinputs) 

print(dense1.dweights) 
print(dense1.dbiases) 
print(dense2.dweights) 
print(dense2.dbiases)

[[0.33333334 0.33333334 0.33333334]
 [0.33331734 0.33331832 0.33336434]
 [0.3332888  0.33329153 0.33341965]
 [0.33325943 0.33326396 0.33347666]
 [0.33323312 0.33323926 0.33352762]]
loss: 1.0984449
acc: 0.34
[[ 0.0014546   0.00082671  0.00045028]
 [ 0.00174065  0.00012617 -0.00035851]]
[[-0.00334526  0.00105437 -0.00096934]]
[[ 0.00048936  0.00102524 -0.0015146 ]
 [-0.00044266 -0.00074882  0.00119148]
 [-0.00054513  0.00084585 -0.00030072]]
[[-0.00106752 -0.00094733  0.0020148 ]]
